In [1]:
%load_ext autoreload
%autoreload 2

# change directory to the root of the project
import os

os.chdir("..")

In [ ]:
from vqpiano.models.factory import model_factory
from omegaconf import OmegaConf
from typing import cast
import torch
from vqpiano.models.segment import SegmentFullSongModel
from safetensors.torch import load_file

model_config = OmegaConf.load("config/simple_ar/model_segment_full_song.yaml").model
model_ = model_factory(model_config)
model = cast(SegmentFullSongModel, model_)
model.load_state_dict(load_file(r"wandb\run-20250513_214542-7xjvhz2d\files\checkpoints\epoch=197-step=1100000.safetensors"))
model.eval()
device = 'cuda:0'
model.to(device)
0

In [ ]:
src

In [ ]:
from pathlib import Path
from music_data_analysis import Pianoroll


# src_path = r"ignore\output\input\ai.mid"

src_path = "ignore/output/input/241221.mid"
# labels = 'ABCBCDCE'
# lengths = [4,8,8,16,8,4,8,8] # bars
labels_and_lengths = 'A4B8C8B16C16D4C8E8'
compose_order = [4, 1,2,3,5,6,7, 0]


# src_path = "ignore/output/input/0814_p.mid"
# labels_and_lengths = 'A4B16C8D8E8B16F8'
# compose_order = [3,2,1,4,5,0,6]

labels = []
lengths = []
cursor = 0
i = 0
while i < len(labels_and_lengths):
    if labels_and_lengths[i].isalpha():
        labels.append(labels_and_lengths[i])
        i += 1
    else:
        # Start collecting digits
        num_str = ""
        while i < len(labels_and_lengths) and labels_and_lengths[i].isdigit():
            num_str += labels_and_lengths[i]
            i += 1
        lengths.append(int(num_str))

src_path = Path(src_path)
src = Pianoroll.from_midi(src_path)
# given_segments = [src]

# given_segments = [src.slice(8*32, 16*32),src.slice(0, 8*32)]


given_segments = []
src_remaining = src.copy()
for i in compose_order:
    length = lengths[i]
    if src_remaining.duration == 0:
        break
    if src_remaining.duration <= length*32:
        given_segments.append(src_remaining)
        break
    given_segments.append(src_remaining[:length*32])
    src_remaining = src_remaining[length*32:]

print('labels', labels)
print('lengths', lengths)
print('compose_order', compose_order)
print('given_segments', given_segments)

name = f'seed_{src_path.stem}'
for i, s in enumerate(given_segments):
    assert s.duration == lengths[compose_order[i]]*32, f'{s.duration} != {lengths[compose_order[i]]*32}'

# given_segments = []
# name = "From Scratch"


n_sample = 10

for i in range(n_sample):

    full_song,annotations = model.sample_song(labels=labels, lengths_in_bars=lengths, compose_order=compose_order, given_segments=given_segments)
    full_song: Pianoroll
    Path(f'ignore/output/{name}').mkdir(parents=True, exist_ok=True)
    # log generated segment
    full_song.to_midi(f'ignore/output/{name}/{i}.mid', markers=annotations)

In [ ]:

full_song.to_midi('ignore/output/test.mid', markers=annotations)

In [ ]:
context_pr.duration

In [ ]:
from music_data_analysis.utils import plt


plt.imshow(composed_segments_sorted[1]["pianoroll"].to_img_tensor())

In [ ]:
from vqpiano.data.factory import dataloader_factory
from omegaconf import OmegaConf

dl = dataloader_factory(
    dataset_config=OmegaConf.load("config/dataset_pop80k_k_pr.yaml"),
    dataloader_config={},
    model_config=OmegaConf.load("config/latent_diff/model_token.yaml"),
)


In [13]:
for i, a in enumerate(dl):
    if i > 30:
        break

In [ ]:
a["pianoroll"][0].to_midi("ignore/output/test.mid")

In [ ]:
a["features"].max()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(a["features"].squeeze(0))

In [ ]:
import torch


plt.imshow((a["features"] + torch.randn_like(a["features"]) * 0.1).squeeze(0)[:, 5:])

In [ ]:
a["pianoroll"][0]